<a href="https://colab.research.google.com/github/sanumalav/CS193_Fall18_Lab1/blob/master/BasicRecommendationLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install turicreate

In [ ]:
import pandas as pd
import numpy as np
import time
import turicreate as tc
from sklearn.model_selection import train_test_split

In [ ]:
import sys
sys.path.append("..")

In [ ]:
# customers  = pd.read_csv('recommend_1.csv')

In [ ]:
# transactions = pd.read_csv('trx_data.csv')

In [ ]:
# customers.head()

In [ ]:
# transactions.head()

In [ ]:

s=time.time()

# data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(),  id_vars=['customerId'], value_name='products').dropna().drop(['variable'], axis=1).groupby(['customerId', 'products']).agg({'products': 'count'}).rename(columns={'products': 'purchase_count'}).reset_index().rename(columns={'products': 'productId'})


In [ ]:
#data.head()

In [ ]:
data2 = pd.read_csv('kz.csv')
data2 = data2.iloc[0:131769]

In [ ]:
len(data2)

131769

In [ ]:
data2.head()

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


In [ ]:
data2 = (data2.groupby(['user_id', 'product_id']).size().reset_index(name='count').drop_duplicates(subset='user_id'))
customers = pd.DataFrame(data2['user_id'].drop_duplicates())
data2.columns = ['customerId', 'productId', 'purchase_count']

KeyError: ignored

In [ ]:
customers = pd.DataFrame(customers, columns = ['customerId'])

In [ ]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data2)
data_dummy


,customerId,productId,purchase_count,purchase_dummy
0,1.515916e+18,1515966223509089059,1,1
1,1.515916e+18,1515966223509089642,1,1
2,1.515916e+18,1515966223509088567,1,1
3,1.515916e+18,1515966223509088620,1,1
11,1.515916e+18,1515966223509107036,1,1
...,...,...,...,...
39073,1.515916e+18,1515966223509089982,1,1
39074,1.515916e+18,1515966223510207823,1,1
39075,1.515916e+18,1515966223509132221,1,1
39077,1.515916e+18,2273948190023614610,1,1


In [ ]:
data_dummy.columns = ['customerId', 'productId', 'purchase_count', 'purchase_dummy']

In [ ]:
df_matrix = pd.pivot_table(data_dummy, values='purchase_count', index='customerId', columns='productId')
df_matrix.head()

productId,1515966223509088493,1515966223509088496,1515966223509088497,1515966223509088498,1515966223509088499,1515966223509088502,1515966223509088503,1515966223509088504,1515966223509088506,1515966223509088508,1515966223509088509,1515966223509088510,1515966223509088514,1515966223509088515,1515966223509088516,1515966223509088519,1515966223509088520,1515966223509088521,1515966223509088522,1515966223509088525,1515966223509088526,1515966223509088528,1515966223509088530,1515966223509088531,1515966223509088532,1515966223509088533,1515966223509088535,1515966223509088536,1515966223509088537,1515966223509088538,1515966223509088541,1515966223509088546,1515966223509088547,1515966223509088548,1515966223509088550,1515966223509088551,1515966223509088552,1515966223509088556,1515966223509088558,1515966223509088560,...,2309018255794111056,2309018256096101325,2309018256263872616,2309018257362780617,2309018259032113499,2309018259040502120,2309018259443155836,2309018260340737007,2309018260391067699,2309018260407844940,2309018263352246672,2309018263696179978,2309018263712957211,2309018263712957212,2309018263712957214,2309018263813620607,2309018263838786470,2309018263922671624,2309018263939448852,2309018264082055352,2309018264107221207,2309018264174330148,2309018264300159434,2309018264300159435,2309018264325325292,2309018264576983696,2309018264576983697,2309018264576983698,2326815104714146556,2326815104722535166,2327084360206910230,2327613725814555347,2332535312850354549,2333247274370466786,2333247274378855397,2333247274437574675,2335421146524025544,2335421146708574974,2335421146742129424,2344947876582392097
customerId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1.515916e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.515916e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.515916e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.515916e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.515916e+18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min()) # normalizing the data #0's and 1's 

In [ ]:
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

# In this step, we have normalized the their purchase history, from 0–1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item).

(9282, 3)


,customerId,productId,scaled_purchase_freq
42893,1.515916e+18,1515966223509088497,0.0
43152,1.515916e+18,1515966223509088497,1.0
43390,1.515916e+18,1515966223509088497,0.0
43731,1.515916e+18,1515966223509088497,0.0
44293,1.515916e+18,1515966223509088497,1.0


#Training and testing data

In [ ]:
def split_data(data):

    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

In [ ]:
train_data, test_data = split_data(data2)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [ ]:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

Baseline Model

In [ ]:
def model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display): #we are using the tc model to calculate the similarity and popularity
    if name == 'popularity':
        model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)
    elif name == 'cosine':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target, 
                                                    similarity_type='pearson')
        
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model

In [ ]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

ToolkitError: ignored